In [1]:
from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "1"
filename1 = './new_data/definition'+name+'_spynorth_scaling_trust.txt'
filename2 = './new_data/definition'+name+'_spynorth_scaling_untrust.txt'
filename3 = './new_data/definition'+name+'_gongjo_scaling_trust.txt'
filename4 = './new_data/definition'+name+'_gongjo_scaling_untrust.txt'
filename5 = './new_data/definition'+name+'_taxi_scaling_trust.txt'
filename6 = './new_data/definition'+name+'_taxi_scaling_untrust.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()
with codecs.open(filename3, 'r', encoding='utf-8-sig') as f:
    lines3 = f.readlines()
with codecs.open(filename4, 'r', encoding='utf-8-sig') as f:
    lines4 = f.readlines()
with codecs.open(filename5, 'r', encoding='utf-8-sig') as f:
    lines5 = f.readlines()
with codecs.open(filename6, 'r', encoding='utf-8-sig') as f:
    lines6 = f.readlines()





# test set


with codecs.open('./new_data/definition'+name+'_spynorth_test_t.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./new_data/definition'+name+'_spynorth_test_ut.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test3 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test4 = f.readlines()
with codecs.open('./new_data/definition'+name+'_taxi_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test5 = f.readlines()
with codecs.open('./new_data/definition'+name+'_taxi_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test6 = f.readlines()


    



In [2]:
from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines3[:2560]
lines_ut = lines4[:2560]
test_t = test3[:640]
test_ut = test4[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:04<00:00, 303.77it/s]


Epoch 1/10
32/32 [==============================] - 21s 569ms/step - loss: 0.6860 - acc: 0.5632 - val_loss: 0.6717 - val_acc: 0.6191
Epoch 2/10
32/32 [==============================] - 21s 676ms/step - loss: 0.6392 - acc: 0.6553 - val_loss: 0.6255 - val_acc: 0.6523
Epoch 3/10
32/32 [==============================] - 23s 743ms/step - loss: 0.5803 - acc: 0.6997 - val_loss: 0.6020 - val_acc: 0.6719
Epoch 4/10
32/32 [==============================] - 20s 642ms/step - loss: 0.5365 - acc: 0.7346 - val_loss: 0.6076 - val_acc: 0.6768
Epoch 5/10
32/32 [==============================] - 17s 521ms/step - loss: 0.5057 - acc: 0.7595 - val_loss: 0.6116 - val_acc: 0.6816
Epoch 6/10
32/32 [==============================] - 21s 656ms/step - loss: 0.4697 - acc: 0.7817 - val_loss: 0.6259 - val_acc: 0.6768
Epoch 7/10
32/32 [==============================] - 17s 520ms/step - loss: 0.4396 - acc: 0.7993 - val_loss: 0.6477 - val_acc: 0.6758
Epoch 8/10
32/32 [==============================] - 21s 672ms/step - 

[0.9552373886108398, 0.550000011920929]

In [3]:
from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1[:2560] + lines3[:2560] + lines5[:2560]
lines_ut = lines2[:2560] + lines4[:2560] + lines6[:2560]
test_t = test1[:640] + test3[:640] + test5[:640]
test_ut = test2[:640] + test4[:640] + test6[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 3840/3840 [00:13<00:00, 275.83it/s]


Epoch 1/10
96/96 [==============================] - 47s 472ms/step - loss: 0.6723 - acc: 0.5922 - val_loss: 0.6428 - val_acc: 0.6426
Epoch 2/10
96/96 [==============================] - 45s 469ms/step - loss: 0.6154 - acc: 0.6675 - val_loss: 0.6385 - val_acc: 0.6442
Epoch 3/10
96/96 [==============================] - 45s 467ms/step - loss: 0.5868 - acc: 0.6941 - val_loss: 0.6425 - val_acc: 0.6442
Epoch 4/10
96/96 [==============================] - 45s 474ms/step - loss: 0.5485 - acc: 0.7210 - val_loss: 0.6570 - val_acc: 0.6380
Epoch 5/10
96/96 [==============================] - 46s 476ms/step - loss: 0.5229 - acc: 0.7403 - val_loss: 0.6811 - val_acc: 0.6393
Epoch 6/10
96/96 [==============================] - 44s 461ms/step - loss: 0.5016 - acc: 0.7586 - val_loss: 0.7000 - val_acc: 0.6283
Epoch 7/10
96/96 [==============================] - 44s 462ms/step - loss: 0.5156 - acc: 0.7625 - val_loss: 0.7301 - val_acc: 0.6276
Epoch 8/10
96/96 [==============================] - 44s 464ms/step - 

[0.9732615351676941, 0.5252603888511658]

In [15]:
from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "2"
filename1 = './new_data/definition'+name+'_spynorth_scaling_trust.txt'
filename2 = './new_data/definition'+name+'_spynorth_scaling_untrust.txt'
filename3 = './new_data/definition'+name+'_gongjo_scaling_trust.txt'
filename4 = './new_data/definition'+name+'_gongjo_scaling_untrust.txt'
filename5 = './new_data/definition'+name+'_taxi_scaling_trust.txt'
filename6 = './new_data/definition'+name+'_taxi_scaling_untrust.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()
with codecs.open(filename3, 'r', encoding='utf-8-sig') as f:
    lines3 = f.readlines()
with codecs.open(filename4, 'r', encoding='utf-8-sig') as f:
    lines4 = f.readlines()
with codecs.open(filename5, 'r', encoding='utf-8-sig') as f:
    lines5 = f.readlines()
with codecs.open(filename6, 'r', encoding='utf-8-sig') as f:
    lines6 = f.readlines()





# test set


with codecs.open('./new_data/definition'+name+'_spynorth_test_t.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./new_data/definition'+name+'_spynorth_test_ut.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test3 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test4 = f.readlines()
with codecs.open('./new_data/definition'+name+'_taxi_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test5 = f.readlines()
with codecs.open('./new_data/definition'+name+'_taxi_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test6 = f.readlines()


    
from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines3[:2560]
lines_ut = lines4[:2560]
test_t = test3[:640]
test_ut = test4[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:02<00:00, 474.99it/s]


Epoch 1/10
32/32 [==============================] - 15s 430ms/step - loss: 0.6879 - acc: 0.5596 - val_loss: 0.6831 - val_acc: 0.5693
Epoch 2/10
32/32 [==============================] - 14s 429ms/step - loss: 0.6678 - acc: 0.5942 - val_loss: 0.6863 - val_acc: 0.5713
Epoch 3/10
32/32 [==============================] - 14s 446ms/step - loss: 0.6398 - acc: 0.6262 - val_loss: 0.6774 - val_acc: 0.5693
Epoch 4/10
32/32 [==============================] - 14s 442ms/step - loss: 0.6018 - acc: 0.6843 - val_loss: 0.6860 - val_acc: 0.5967
Epoch 5/10
32/32 [==============================] - 14s 448ms/step - loss: 0.5566 - acc: 0.7239 - val_loss: 0.7111 - val_acc: 0.5918
Epoch 6/10
32/32 [==============================] - 14s 423ms/step - loss: 0.5176 - acc: 0.7507 - val_loss: 0.7456 - val_acc: 0.5771
Epoch 7/10
32/32 [==============================] - 15s 472ms/step - loss: 0.4786 - acc: 0.7664 - val_loss: 0.7703 - val_acc: 0.5703
Epoch 8/10
32/32 [==============================] - 14s 424ms/step - 

[0.9941390156745911, 0.512499988079071]

In [5]:
from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1[:2560] + lines3[:2560] + lines5[:2560]
lines_ut = lines2[:2560] + lines4[:2560] + lines6[:2560]
test_t = test1[:640] + test3[:640] + test5[:640]
test_ut = test2[:640] + test4[:640] + test6[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 3840/3840 [00:10<00:00, 376.74it/s]


Epoch 1/10
96/96 [==============================] - 41s 420ms/step - loss: 0.6871 - acc: 0.5494 - val_loss: 0.6776 - val_acc: 0.5742
Epoch 2/10
96/96 [==============================] - 40s 412ms/step - loss: 0.6608 - acc: 0.6088 - val_loss: 0.6734 - val_acc: 0.5889
Epoch 3/10
96/96 [==============================] - 40s 415ms/step - loss: 0.6258 - acc: 0.6559 - val_loss: 0.6858 - val_acc: 0.5745
Epoch 4/10
96/96 [==============================] - 40s 414ms/step - loss: 0.5833 - acc: 0.6892 - val_loss: 0.7223 - val_acc: 0.5758
Epoch 5/10
96/96 [==============================] - 40s 416ms/step - loss: 0.5484 - acc: 0.7151 - val_loss: 0.7456 - val_acc: 0.5563
Epoch 6/10
96/96 [==============================] - 40s 422ms/step - loss: 0.5169 - acc: 0.7359 - val_loss: 0.7833 - val_acc: 0.5560
Epoch 7/10
96/96 [==============================] - 41s 424ms/step - loss: 0.5066 - acc: 0.7482 - val_loss: 0.8134 - val_acc: 0.5583
Epoch 8/10
96/96 [==============================] - 41s 426ms/step - 

[0.9955365657806396, 0.49010416865348816]

In [6]:
from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
filename1 = './new_data/definition'+name+'_spynorth_scaling_trust.txt'
filename2 = './new_data/definition'+name+'_spynorth_scaling_untrust.txt'
filename3 = './new_data/definition'+name+'_gongjo_scaling_trust.txt'
filename4 = './new_data/definition'+name+'_gongjo_scaling_untrust.txt'
filename5 = './new_data/definition'+name+'_taxi_scaling_trust.txt'
filename6 = './new_data/definition'+name+'_taxi_scaling_untrust.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()
with codecs.open(filename3, 'r', encoding='utf-8-sig') as f:
    lines3 = f.readlines()
with codecs.open(filename4, 'r', encoding='utf-8-sig') as f:
    lines4 = f.readlines()
with codecs.open(filename5, 'r', encoding='utf-8-sig') as f:
    lines5 = f.readlines()
with codecs.open(filename6, 'r', encoding='utf-8-sig') as f:
    lines6 = f.readlines()





# test set


with codecs.open('./new_data/definition'+name+'_spynorth_test_t.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./new_data/definition'+name+'_spynorth_test_ut.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test3 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test4 = f.readlines()
with codecs.open('./new_data/definition'+name+'_taxi_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test5 = f.readlines()
with codecs.open('./new_data/definition'+name+'_taxi_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test6 = f.readlines()


    
from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines3[:2560]
lines_ut = lines4[:2560]
test_t = test3[:640]
test_ut = test4[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:03<00:00, 406.53it/s]


Epoch 1/10
32/32 [==============================] - 15s 430ms/step - loss: 0.6935 - acc: 0.4993 - val_loss: 0.6928 - val_acc: 0.5479
Epoch 2/10
32/32 [==============================] - 14s 426ms/step - loss: 0.6896 - acc: 0.5747 - val_loss: 0.6916 - val_acc: 0.5254
Epoch 3/10
32/32 [==============================] - 13s 419ms/step - loss: 0.6734 - acc: 0.6035 - val_loss: 0.6922 - val_acc: 0.5322
Epoch 4/10
32/32 [==============================] - 13s 419ms/step - loss: 0.6361 - acc: 0.6543 - val_loss: 0.7086 - val_acc: 0.5127
Epoch 5/10
32/32 [==============================] - 13s 417ms/step - loss: 0.5837 - acc: 0.7043 - val_loss: 0.7402 - val_acc: 0.5176
Epoch 6/10
32/32 [==============================] - 13s 415ms/step - loss: 1.2529 - acc: 0.5784 - val_loss: 1.4315 - val_acc: 0.5020
Epoch 7/10
32/32 [==============================] - 13s 418ms/step - loss: 1.0879 - acc: 0.5491 - val_loss: 1.0826 - val_acc: 0.5244
Epoch 8/10
32/32 [==============================] - 13s 415ms/step - 

[0.8789447546005249, 0.5375000238418579]

In [7]:
from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1[:2560] + lines3[:2560] + lines5[:2560]
lines_ut = lines2[:2560] + lines4[:2560] + lines6[:2560]
test_t = test1[:640] + test3[:640] + test5[:640]
test_ut = test2[:640] + test4[:640] + test6[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 3840/3840 [00:10<00:00, 361.70it/s]


Epoch 1/10
96/96 [==============================] - 42s 423ms/step - loss: 0.6929 - acc: 0.5149 - val_loss: 0.6925 - val_acc: 0.5117
Epoch 2/10
96/96 [==============================] - 40s 418ms/step - loss: 0.6818 - acc: 0.5714 - val_loss: 0.6908 - val_acc: 0.5270
Epoch 3/10
96/96 [==============================] - 40s 417ms/step - loss: 0.6487 - acc: 0.6233 - val_loss: 0.7055 - val_acc: 0.5267
Epoch 4/10
96/96 [==============================] - 40s 418ms/step - loss: 0.6098 - acc: 0.6669 - val_loss: 0.7353 - val_acc: 0.5189
Epoch 5/10
96/96 [==============================] - 40s 417ms/step - loss: 1.4037 - acc: 0.5754 - val_loss: 0.8253 - val_acc: 0.4990
Epoch 6/10
96/96 [==============================] - 39s 411ms/step - loss: 0.6108 - acc: 0.6625 - val_loss: 0.7604 - val_acc: 0.5150
Epoch 7/10
96/96 [==============================] - 39s 404ms/step - loss: 0.5509 - acc: 0.7082 - val_loss: 0.8027 - val_acc: 0.5163
Epoch 8/10
96/96 [==============================] - 39s 410ms/step - 

[0.9784433841705322, 0.504687488079071]

# 완벽한 타인 + 공조 + 공작

In [8]:
from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "1"
filename1 = './new_data/definition'+name+'_spynorth_scaling_trust.txt'
filename2 = './new_data/definition'+name+'_spynorth_scaling_untrust.txt'
filename3 = './new_data/definition'+name+'_gongjo_scaling_trust.txt'
filename4 = './new_data/definition'+name+'_gongjo_scaling_untrust.txt'
filename5 = './new_data/definition'+name+'_intistranger_scaling_trust.txt'
filename6 = './new_data/definition'+name+'_intistranger_scaling_untrust.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()
with codecs.open(filename3, 'r', encoding='utf-8-sig') as f:
    lines3 = f.readlines()
with codecs.open(filename4, 'r', encoding='utf-8-sig') as f:
    lines4 = f.readlines()
with codecs.open(filename5, 'r', encoding='utf-8-sig') as f:
    lines5 = f.readlines()
with codecs.open(filename6, 'r', encoding='utf-8-sig') as f:
    lines6 = f.readlines()





# test set


with codecs.open('./new_data/definition'+name+'_spynorth_test_t.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./new_data/definition'+name+'_spynorth_test_ut.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test3 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test4 = f.readlines()
with codecs.open('./new_data/definition'+name+'_intistranger_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test5 = f.readlines()
with codecs.open('./new_data/definition'+name+'_intistranger_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test6 = f.readlines()


    

from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1[:2560] + lines3[:2560] + lines5[:2560]
lines_ut = lines2[:2560] + lines4[:2560] + lines6[:2560]
test_t = test1[:640] + test3[:640] + test5[:640]
test_ut = test2[:640] + test4[:640] + test6[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 3840/3840 [00:10<00:00, 365.23it/s]


Epoch 1/10
96/96 [==============================] - 42s 427ms/step - loss: 0.6753 - acc: 0.5780 - val_loss: 0.6587 - val_acc: 0.6110
Epoch 2/10
96/96 [==============================] - 41s 427ms/step - loss: 0.6184 - acc: 0.6619 - val_loss: 0.6408 - val_acc: 0.6452
Epoch 3/10
96/96 [==============================] - 41s 425ms/step - loss: 0.5816 - acc: 0.6981 - val_loss: 0.6464 - val_acc: 0.6436
Epoch 4/10
96/96 [==============================] - 41s 425ms/step - loss: 0.5823 - acc: 0.7074 - val_loss: 0.6621 - val_acc: 0.6276
Epoch 5/10
96/96 [==============================] - 41s 424ms/step - loss: 0.5315 - acc: 0.7410 - val_loss: 0.6876 - val_acc: 0.6273
Epoch 6/10
96/96 [==============================] - 41s 423ms/step - loss: 0.5126 - acc: 0.7517 - val_loss: 0.6966 - val_acc: 0.6169
Epoch 7/10
96/96 [==============================] - 41s 424ms/step - loss: 0.4971 - acc: 0.7631 - val_loss: 0.7053 - val_acc: 0.6126
Epoch 8/10
96/96 [==============================] - 40s 418ms/step - 

[0.9339373111724854, 0.5179687738418579]

In [9]:
from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "2"
filename1 = './new_data/definition'+name+'_spynorth_scaling_trust.txt'
filename2 = './new_data/definition'+name+'_spynorth_scaling_untrust.txt'
filename3 = './new_data/definition'+name+'_gongjo_scaling_trust.txt'
filename4 = './new_data/definition'+name+'_gongjo_scaling_untrust.txt'
filename5 = './new_data/definition'+name+'_intistranger_scaling_trust.txt'
filename6 = './new_data/definition'+name+'_intistranger_scaling_untrust.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()
with codecs.open(filename3, 'r', encoding='utf-8-sig') as f:
    lines3 = f.readlines()
with codecs.open(filename4, 'r', encoding='utf-8-sig') as f:
    lines4 = f.readlines()
with codecs.open(filename5, 'r', encoding='utf-8-sig') as f:
    lines5 = f.readlines()
with codecs.open(filename6, 'r', encoding='utf-8-sig') as f:
    lines6 = f.readlines()





# test set


with codecs.open('./new_data/definition'+name+'_spynorth_test_t.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./new_data/definition'+name+'_spynorth_test_ut.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test3 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test4 = f.readlines()
with codecs.open('./new_data/definition'+name+'_intistranger_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test5 = f.readlines()
with codecs.open('./new_data/definition'+name+'_intistranger_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test6 = f.readlines()


    

from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1[:2560] + lines3[:2560] + lines5[:2560]
lines_ut = lines2[:2560] + lines4[:2560] + lines6[:2560]
test_t = test1[:640] + test3[:640] + test5[:640]
test_ut = test2[:640] + test4[:640] + test6[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 3840/3840 [00:09<00:00, 388.45it/s]


Epoch 1/10
96/96 [==============================] - 41s 414ms/step - loss: 0.6859 - acc: 0.5507 - val_loss: 0.6801 - val_acc: 0.5641
Epoch 2/10
96/96 [==============================] - 40s 412ms/step - loss: 0.6634 - acc: 0.6007 - val_loss: 0.6752 - val_acc: 0.5677
Epoch 3/10
96/96 [==============================] - 39s 410ms/step - loss: 0.6303 - acc: 0.6492 - val_loss: 0.6890 - val_acc: 0.5716
Epoch 4/10
96/96 [==============================] - 40s 420ms/step - loss: 0.5935 - acc: 0.6793 - val_loss: 0.7147 - val_acc: 0.5589
Epoch 5/10
96/96 [==============================] - 41s 424ms/step - loss: 0.5562 - acc: 0.7058 - val_loss: 0.7443 - val_acc: 0.5518
Epoch 6/10
96/96 [==============================] - 40s 422ms/step - loss: 0.5258 - acc: 0.7326 - val_loss: 0.7985 - val_acc: 0.5544
Epoch 7/10
96/96 [==============================] - 41s 426ms/step - loss: 0.4983 - acc: 0.7480 - val_loss: 0.8168 - val_acc: 0.5410
Epoch 8/10
96/96 [==============================] - 41s 424ms/step - 

[0.9211099743843079, 0.5335937738418579]

In [10]:
from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
filename1 = './new_data/definition'+name+'_spynorth_scaling_trust.txt'
filename2 = './new_data/definition'+name+'_spynorth_scaling_untrust.txt'
filename3 = './new_data/definition'+name+'_gongjo_scaling_trust.txt'
filename4 = './new_data/definition'+name+'_gongjo_scaling_untrust.txt'
filename5 = './new_data/definition'+name+'_intistranger_scaling_trust.txt'
filename6 = './new_data/definition'+name+'_intistranger_scaling_untrust.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()
with codecs.open(filename3, 'r', encoding='utf-8-sig') as f:
    lines3 = f.readlines()
with codecs.open(filename4, 'r', encoding='utf-8-sig') as f:
    lines4 = f.readlines()
with codecs.open(filename5, 'r', encoding='utf-8-sig') as f:
    lines5 = f.readlines()
with codecs.open(filename6, 'r', encoding='utf-8-sig') as f:
    lines6 = f.readlines()





# test set


with codecs.open('./new_data/definition'+name+'_spynorth_test_t.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./new_data/definition'+name+'_spynorth_test_ut.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test3 = f.readlines()
with codecs.open('./new_data/definition'+name+'_gongjo_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test4 = f.readlines()
with codecs.open('./new_data/definition'+name+'_intistranger_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test5 = f.readlines()
with codecs.open('./new_data/definition'+name+'_intistranger_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test6 = f.readlines()


    

from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1[:2560] + lines3[:2560] + lines5[:2560]
lines_ut = lines2[:2560] + lines4[:2560] + lines6[:2560]
test_t = test1[:640] + test3[:640] + test5[:640]
test_ut = test2[:640] + test4[:640] + test6[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 3840/3840 [00:10<00:00, 371.86it/s]


Epoch 1/10
96/96 [==============================] - 42s 424ms/step - loss: 0.6928 - acc: 0.5129 - val_loss: 0.6919 - val_acc: 0.5257
Epoch 2/10
96/96 [==============================] - 41s 428ms/step - loss: 0.6781 - acc: 0.5833 - val_loss: 0.6904 - val_acc: 0.5433
Epoch 3/10
96/96 [==============================] - 41s 425ms/step - loss: 0.6461 - acc: 0.6297 - val_loss: 0.7072 - val_acc: 0.5319
Epoch 4/10
96/96 [==============================] - 41s 427ms/step - loss: 0.6098 - acc: 0.6658 - val_loss: 0.7344 - val_acc: 0.5267
Epoch 5/10
96/96 [==============================] - 41s 428ms/step - loss: 0.5805 - acc: 0.6895 - val_loss: 0.7645 - val_acc: 0.5202
Epoch 6/10
96/96 [==============================] - 41s 423ms/step - loss: 1.0152 - acc: 0.6332 - val_loss: 0.7689 - val_acc: 0.5273
Epoch 7/10
96/96 [==============================] - 39s 409ms/step - loss: 0.5429 - acc: 0.7246 - val_loss: 0.7955 - val_acc: 0.5257
Epoch 8/10
96/96 [==============================] - 39s 407ms/step - 

[0.9145938754081726, 0.5013020634651184]

In [11]:
from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "1"
filename1 = './helpfulness/spynorth_scaling_trust.txt'
filename2 = './helpfulness/spynorth_scaling_untrust.txt'
filename3 = './helpfulness/gongjo_scaling_trust.txt'
filename4 = './helpfulness/gongjo_scaling_untrust.txt'
filename5 = './helpfulness/taxi_scaling_trust.txt'
filename6 = './helpfulness/taxi_scaling_untrust.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()
with codecs.open(filename3, 'r', encoding='utf-8-sig') as f:
    lines3 = f.readlines()
with codecs.open(filename4, 'r', encoding='utf-8-sig') as f:
    lines4 = f.readlines()
with codecs.open(filename5, 'r', encoding='utf-8-sig') as f:
    lines5 = f.readlines()
with codecs.open(filename6, 'r', encoding='utf-8-sig') as f:
    lines6 = f.readlines()





# test set


with codecs.open('./helpfulness/spynorth_test_t.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./helpfulness/spynorth_test_ut.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()
with codecs.open('./helpfulness/gongjo_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test3 = f.readlines()
with codecs.open('./helpfulness/gongjo_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test4 = f.readlines()
with codecs.open('./helpfulness/taxi_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test5 = f.readlines()
with codecs.open('./helpfulness/taxi_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test6 = f.readlines()


    
from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines3[:2560]
lines_ut = lines4[:2560]
test_t = test3[:640]
test_ut = test4[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 1280/1280 [00:03<00:00, 395.26it/s]


Epoch 1/10
32/32 [==============================] - 15s 437ms/step - loss: 0.6885 - acc: 0.5566 - val_loss: 0.6777 - val_acc: 0.6055
Epoch 2/10
32/32 [==============================] - 14s 428ms/step - loss: 0.6576 - acc: 0.6248 - val_loss: 0.6480 - val_acc: 0.6387
Epoch 3/10
32/32 [==============================] - 14s 425ms/step - loss: 0.6143 - acc: 0.6753 - val_loss: 0.6454 - val_acc: 0.6162
Epoch 4/10
32/32 [==============================] - 14s 428ms/step - loss: 0.5722 - acc: 0.7051 - val_loss: 0.6598 - val_acc: 0.6143
Epoch 5/10
32/32 [==============================] - 14s 427ms/step - loss: 0.5300 - acc: 0.7412 - val_loss: 0.6778 - val_acc: 0.6045
Epoch 6/10
32/32 [==============================] - 14s 427ms/step - loss: 0.5088 - acc: 0.7646 - val_loss: 0.6970 - val_acc: 0.6084
Epoch 7/10
32/32 [==============================] - 14s 429ms/step - loss: 0.4692 - acc: 0.7844 - val_loss: 0.7289 - val_acc: 0.6016
Epoch 8/10
32/32 [==============================] - 14s 430ms/step - 

[1.0045051574707031, 0.50390625]

In [12]:
from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1[:2560] + lines3[:2560] + lines5[:2560]
lines_ut = lines2[:2560] + lines4[:2560] + lines6[:2560]
test_t = test1[:640] + test3[:640] + test5[:640]
test_ut = test2[:640] + test4[:640] + test6[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 3840/3840 [00:10<00:00, 358.26it/s]


Epoch 1/10
96/96 [==============================] - 43s 434ms/step - loss: 0.6898 - acc: 0.5299 - val_loss: 0.6860 - val_acc: 0.5472
Epoch 2/10
96/96 [==============================] - 41s 429ms/step - loss: 0.6601 - acc: 0.6070 - val_loss: 0.6899 - val_acc: 0.5505
Epoch 3/10
96/96 [==============================] - 41s 429ms/step - loss: 0.6179 - acc: 0.6552 - val_loss: 0.7001 - val_acc: 0.5511
Epoch 4/10
96/96 [==============================] - 41s 428ms/step - loss: 0.5794 - acc: 0.6937 - val_loss: 0.7181 - val_acc: 0.5531
Epoch 5/10
96/96 [==============================] - 41s 428ms/step - loss: 0.5457 - acc: 0.7131 - val_loss: 0.7624 - val_acc: 0.5531
Epoch 6/10
96/96 [==============================] - 41s 427ms/step - loss: 0.5143 - acc: 0.7381 - val_loss: 0.8306 - val_acc: 0.5573
Epoch 7/10
96/96 [==============================] - 41s 428ms/step - loss: 0.7546 - acc: 0.6887 - val_loss: 0.8062 - val_acc: 0.5488
Epoch 8/10
96/96 [==============================] - 41s 427ms/step - 

[0.81831955909729, 0.49166667461395264]

In [13]:
from keras.preprocessing.text import Tokenizer
import codecs
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import nltk
from konlpy.tag import Okt
from tqdm import tqdm

# train set
name = "3"
filename1 = './helpfulness/spynorth_scaling_trust.txt'
filename2 = './helpfulness/spynorth_scaling_untrust.txt'
filename3 = './helpfulness/gongjo_scaling_trust.txt'
filename4 = './helpfulness/gongjo_scaling_untrust.txt'
filename5 = './helpfulness/intistranger_scaling_trust.txt'
filename6 = './helpfulness/intistranger_scaling_untrust.txt'



with codecs.open(filename1, 'r', encoding='utf-8-sig') as f:
    lines1 = f.readlines()
with codecs.open(filename2, 'r', encoding='utf-8-sig') as f:
    lines2 = f.readlines()
with codecs.open(filename3, 'r', encoding='utf-8-sig') as f:
    lines3 = f.readlines()
with codecs.open(filename4, 'r', encoding='utf-8-sig') as f:
    lines4 = f.readlines()
with codecs.open(filename5, 'r', encoding='utf-8-sig') as f:
    lines5 = f.readlines()
with codecs.open(filename6, 'r', encoding='utf-8-sig') as f:
    lines6 = f.readlines()





# test set


with codecs.open('./helpfulness/spynorth_test_t.txt', 'r', 'utf-8-sig') as f:
    test1 = f.readlines()
with codecs.open('./helpfulness/spynorth_test_ut.txt', 'r', 'utf-8-sig') as f:
    test2 = f.readlines()
with codecs.open('./helpfulness/gongjo_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test3 = f.readlines()
with codecs.open('./helpfulness/gongjo_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test4 = f.readlines()
with codecs.open('./helpfulness/intistranger_test_t.txt', 'r', encoding='utf-8-sig') as f:
    test5 = f.readlines()
with codecs.open('./helpfulness/intistranger_test_ut.txt', 'r', encoding='utf-8-sig') as f:
    test6 = f.readlines()


    

from keras.layers import Input, Dense
from keras.models import Model
lines_t = lines1[:2560] + lines3[:2560] + lines5[:2560]
lines_ut = lines2[:2560] + lines4[:2560] + lines6[:2560]
test_t = test1[:640] + test3[:640] + test5[:640]
test_ut = test2[:640] + test4[:640] + test6[:640]


lines_ = []
test_lines_ = []

lines1_ = []
lines2_ = []
lines3_ = []
lines4_ = []
feature1 = []
feature2 = []
rating = []
sentiment = []
correlation = []

for line in lines_t:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)
    
    

for line in lines_ut:
    text = line.split(",")[3]
    lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature1.append(a)

for line in test_t:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
for line in test_ut:
    text = line.split(",")[3]
    test_lines_.append(text.strip())
    a = line.split(",")[:3]
    a = list(map(float, a))
    feature2.append(a)
    
    
train_labels = [] # train 데이터 label
test_labels = [] # test 데이터 label
for i in range(len(lines_t)):
    train_labels.append(1)
for j in range(len(lines_ut)):
    train_labels.append(0)
for i in range(len(test_t)):
    test_labels.append(1)
for j in range(len(test_ut)):
    test_labels.append(0)

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

okt = Okt()
x_train = []
x_test = []
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

# train data
for sentence in tqdm(lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_train.append(stopwords_removed_sentence)

# test data
for sentence in tqdm(test_lines_):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    x_test.append(stopwords_removed_sentence)

maxlen = 1000  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200 -> 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다
# train 토큰화
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

word_index = tokenizer.word_index
train = pad_sequences(sequences,maxlen=maxlen)

# test 토큰화
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(x_test)
sequences2 = tokenizer2.texts_to_sequences(x_test)

word_index2 = tokenizer2.word_index
test = pad_sequences(sequences2,maxlen=maxlen)


from keras.preprocessing import sequence
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 10000  # 특성으로 사용할 단어의 수

batch_size = 32
input_train = sequence.pad_sequences(train,maxlen=maxlen)
input_test = sequence.pad_sequences(test,maxlen=maxlen)




from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(input_train, train_labels, test_size=0.2, shuffle=True, stratify=train_labels, random_state=34)
# LSTM
from keras.layers import LSTM
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN

from keras.layers import Dense

from keras.preprocessing import sequence

model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
# 'binary_crossentropy
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

from keras.callbacks import EarlyStopping
#early_stopping = EarlyStopping(monitor='val_acc', patience=10, mode='auto')

"""history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=100,
                    batch_size=100,callbacks=[early_stopping]
                    )"""

history = model.fit(x_train, y_train,validation_data=(x_valid, y_valid),
                    epochs=10,
                    batch_size=128
                    )

model.evaluate(input_test, test_labels)

100%|█████████████████████████████████████████████████████████████████████████████| 3840/3840 [00:10<00:00, 368.31it/s]


Epoch 1/10
96/96 [==============================] - 42s 429ms/step - loss: 0.6881 - acc: 0.5439 - val_loss: 0.6816 - val_acc: 0.5690
Epoch 2/10
96/96 [==============================] - 41s 423ms/step - loss: 0.6563 - acc: 0.6216 - val_loss: 0.6784 - val_acc: 0.5785
Epoch 3/10
96/96 [==============================] - 40s 413ms/step - loss: 0.6204 - acc: 0.6618 - val_loss: 0.6913 - val_acc: 0.5879
Epoch 4/10
96/96 [==============================] - 40s 412ms/step - loss: 0.6380 - acc: 0.6772 - val_loss: 0.7009 - val_acc: 0.5732
Epoch 5/10
96/96 [==============================] - 39s 412ms/step - loss: 0.5972 - acc: 0.6919 - val_loss: 0.7343 - val_acc: 0.5703
Epoch 6/10
96/96 [==============================] - 39s 404ms/step - loss: 0.5433 - acc: 0.7248 - val_loss: 0.7544 - val_acc: 0.5697
Epoch 7/10
96/96 [==============================] - 39s 404ms/step - loss: 0.5198 - acc: 0.7411 - val_loss: 0.7814 - val_acc: 0.5622
Epoch 8/10
96/96 [==============================] - 39s 411ms/step - 

[0.9747170805931091, 0.4958333373069763]

In [14]:
print(model.metrics_names)

['loss', 'acc']
